In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/deepfake_detection


/content/drive/MyDrive/deepfake_detection


In [ ]:
!pip install torch numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import numpy as np

v1 = np.load("/content/drive/MyDrive/deepfake_detection/data/all_embeddings.npy")
v2 = np.load("/content/drive/MyDrive/deepfake_detection/data/v2_embeddings.npy")
combined = np.concatenate((v1, v2), axis=1)
np.save("/content/drive/MyDrive/deepfake_detection/data/combined_embeddings.npy", combined)

# Save labels (use v2 or all)
labels = np.load("/content/drive/MyDrive/deepfake_detection/data/v2_labels.npy")
np.save("/content/drive/MyDrive/deepfake_detection/data/combined_labels.npy", labels)


In [ ]:
%%writefile /content/drive/MyDrive/deepfake_detection/sho_simple.py
import numpy as np

def SHO(obj_func, lb, ub, num_hyenas=50, max_iter=100):
    dim = len(lb)
    hyenas = np.random.uniform(lb, ub, (num_hyenas, dim))
    fitness = np.zeros(num_hyenas)

    for t in range(max_iter):
        for i in range(num_hyenas):
            fitness[i] = obj_func(hyenas[i, :])

        best_hyena_idx = np.argmin(fitness)
        best_hyena = hyenas[best_hyena_idx, :]

        for i in range(num_hyenas):
            if i == best_hyena_idx:
                continue

            dist = np.linalg.norm(hyenas[i, :] - best_hyena)
            if fitness[i] < fitness[best_hyena_idx]:
                a = 2.0 - 2.0 * t / max_iter
                b = np.random.uniform(0, 1)
                hyenas[i, :] += a * np.exp(-b * dist) * (hyenas[i, :] - best_hyena)
            else:
                a = 2.0 * t / max_iter
                b = np.random.uniform(0, 1)
                hyenas[i, :] += a * np.exp(-b * dist) * (hyenas[i, :] - best_hyena)

            hyenas[i, :] = np.clip(hyenas[i, :], lb, ub)

    best_idx = np.argmin(fitness)
    return hyenas[best_idx, :], fitness[best_idx]


Overwriting /content/drive/MyDrive/deepfake_detection/sho_simple.py


In [ ]:
!python /content/drive/MyDrive/deepfake_detection/train_task2_enhanced_sho.py


Traceback (most recent call last):
  File "/content/drive/MyDrive/deepfake_detection/train_task2_enhanced_sho.py", line 111, in <module>
    best_solution, best_fitness = SHO(fitness, lb, ub, num_hyenas=10, max_iter=60, verbose=True, callback=sho_callback)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: SHO() got an unexpected keyword argument 'verbose'


In [ ]:
!python /content/drive/MyDrive/deepfake_detection/task3_updated.py


✅ Model loaded. Starting evaluation on test set...

🎯 Test Set Evaluation Results:
Accuracy : 0.5149
Precision: 0.5573
Recall   : 0.4280
F1 Score : 0.4842
📁 Video-level predictions saved to task3_video_results_enhanced.csv


In [ ]:
import os

print(os.listdir("/content/drive/MyDrive/deepfake_detection"))


['data', 'models', '__pycache__', '.ipynb_checkpoints', 'train_task2_enhanced_sho.py', 'mem_task2_bilstm_sho_enhanced.pth', 'task3_video_results_enhanced.csv', 'task3_updated.py', 'train_task2_adam.py', 'task3_adam.py', 'sho_simple.py', 'mem_task2_bilstm_adam.pth', 'task3_video_results_adam.csv']


In [ ]:
!python /content/drive/MyDrive/deepfake_detection/train_task2_adam.py


Epoch 1/10, Loss: 0.6103
Epoch 2/10, Loss: 0.2600
Epoch 3/10, Loss: 0.1067
Epoch 4/10, Loss: 0.0525
Epoch 5/10, Loss: 0.0349
Epoch 6/10, Loss: 0.0103
Epoch 7/10, Loss: 0.0275
Epoch 8/10, Loss: 0.0484
Epoch 9/10, Loss: 0.0086
Epoch 10/10, Loss: 0.0215
✅ Model trained using Adam and saved successfully!


In [ ]:
!python /content/drive/MyDrive/deepfake_detection/task3_adam.py


✅ Adam-trained model loaded. Starting evaluation on test set...

🎯 Adam Model Evaluation Results:
Accuracy : 0.9809
Precision: 0.9725
Recall   : 0.9920
F1 Score : 0.9822
📁 Adam video-level predictions saved to task3_video_results_adam.csv


In [ ]:
#Class Distribution of Labels
import numpy as np
labels = np.load("/content/drive/MyDrive/deepfake_detection/data/combined_labels.npy")
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))


{np.int64(0): np.int64(8796), np.int64(1): np.int64(10000)}


In [ ]:
# Distribution of Test Labels
test_labels = np.load("/content/drive/MyDrive/deepfake_detection/data/test_labels.npy")
unique, counts = np.unique(test_labels, return_counts=True)
print("Test set label distribution:", dict(zip(unique, counts)))


Test set label distribution: {np.int64(0): np.int64(220), np.int64(1): np.int64(250)}


In [ ]:
# Model Predictions
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/deepfake_detection/task3_video_results_adam.csv")
print(df['Predicted_Label'].value_counts())


Predicted_Label
1    255
0    215
Name: count, dtype: int64


In [ ]:
!ls /content/drive/MyDrive/deepfake_detection/


data				   task3_adam.py
mem_task2_bilstm_adam.pth	   task3_updated.py
mem_task2_bilstm_sho_enhanced.pth  task3_video_results_adam.csv
models				   task3_video_results_enhanced.csv
__pycache__			   train_task2_adam.py
sho_simple.py			   train_task2_enhanced_sho.py


In [ ]:
# Load all dependencies in one cell
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Add the parent directory of 'models' to the system path
# Assuming 'models' is directly inside '/content/drive/MyDrive/deepfake_detection/'
import sys
import os
models_dir = "/content/drive/MyDrive/deepfake_detection/"
if models_dir not in sys.path:
    sys.path.append(models_dir)

# Now, the import should work if the 'models' directory and 'mem_task2_model.py' exist within models_dir
from models.mem_task2_model import BiLSTMAttentionModel


# 1. Load and preprocess data (same as train_task2_adam.py)
all_features = np.load("/content/drive/MyDrive/deepfake_detection/data/combined_embeddings.npy")
all_labels = np.load("/content/drive/MyDrive/deepfake_detection/data/combined_labels.npy")
total_frames = (len(all_features) // 8) * 8
train_features = all_features[:total_frames].reshape(-1, 8, 1024)
train_labels = all_labels[:total_frames].reshape(-1, 8)
majority_labels = np.array([np.bincount(seq).argmax() for seq in train_labels])

# 2. Recreate Dataset/Dataloader (same as original)
class FeatureSequenceDataset(Dataset):
    def __init__(self, features, labels, sequence_length=8):
        self.features = features.reshape(-1, 1024)
        # The labels should also be shaped to match the number of sequences, not the individual frames
        self.labels = labels
        self.sequence_length = sequence_length
        self.num_sequences = len(self.features) // sequence_length

    def __len__(self):
        # Added __len__ method to return the number of sequences
        return self.num_sequences

    def __getitem__(self, idx):
        start = idx * self.sequence_length
        end = start + self.sequence_length
        sequence = self.features[start:end]
        # Get the single majority label for the sequence
        label = self.labels[idx]
        return torch.tensor(sequence, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Pass the majority labels (one per sequence) to the dataset
train_dataset = FeatureSequenceDataset(train_features, majority_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# 3. Load trained model (same as task3_adam.py)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMAttentionModel(input_dim=1024, num_classes=2).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/deepfake_detection/mem_task2_bilstm_adam.pth", map_location=device))

# 4. Calculate training accuracy
model.eval()
correct = total = 0
with torch.no_grad():
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Training Accuracy: {100 * correct / total:.2f}%")

Training Accuracy: 99.02%
